In [166]:
import pandas as pd
patients = pd.read_csv("data/PATIENTS.csv")
dicd = pd.read_csv("data/D_ICD_DIAGNOSES.csv")
diagnoses = pd.read_csv("data/DIAGNOSES_ICD.csv")
patients["gender"].unique()

array(['F', 'M'], dtype=object)

In [167]:
#4a
gender_counts = patients["gender"].value_counts()
gender_counts

gender
F    55
M    45
Name: count, dtype: int64

In [168]:
#4b
def get_subject_ids(diagnosis: str) -> list:
    icd9 = dicd.loc[dicd["long_title"] == diagnosis, "icd9_code"].unique()
    subject_ids = diagnoses.loc[diagnoses["icd9_code"].isin(icd9), "subject_id"].unique()
    return  [int(x) for x in subject_ids]

In [169]:
#4c
# 1) Known diagnosis: should return a non-empty list
name1 = "Intestinal infection due to Clostridium difficile"
result1 = get_subject_ids(name1)
print("Result for valid diagnosis_1:", result1)
assert isinstance(result1, list)

name2 = "Down's syndrome" # manual check: should return only one id 42066
result2 = get_subject_ids(name2)
print("Result for valid diagnosis_2:", result2)
assert isinstance(result2, list)

# 2) Known diagnosis: should return empty since no subject got the syndrome  
name3 = "Tuberculosis of ureter, unspecified"
result3 = get_subject_ids(name3)
print("Result for valid diagnosis_3:", result3)
assert result3 == []

# 3) A D.N.E diagnosis: should return empty
name4 = "nothing"
result4 = get_subject_ids(name4)
print("Result for D.N.E diagnosis:", result4)
assert result4 == []

# 4) Test no duplicates occur 
assert len(result1) == len(set(result1))

# 5) Test all int ids
assert all(isinstance(x, int) for x in result1)

# 6) Test every returned subject_id should exist in PATIENTS.
patient_ids = set(patients["subject_id"].astype(int))
assert set(result1).issubset(patient_ids), "Got IDs not present in PATIENTS"
assert set(result2).issubset(patient_ids), "Got IDs not present in PATIENTS"

Result for valid diagnosis_1: [10043, 10045, 10094, 10102, 40595, 41976, 44228]
Result for valid diagnosis_2: [42066]
Result for valid diagnosis_3: []
Result for D.N.E diagnosis: []


In [170]:
#4d
cd_ids = get_subject_ids("Intestinal infection due to Clostridium difficile")
cd_patients = patients[patients["subject_id"].isin(cd_ids)].copy()
cd_patients["dob"] = pd.to_datetime(cd_patients["dob"])
cd_patients["dod"] = pd.to_datetime(cd_patients["dod"])
cd_patients

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
15,9502,10043,M,2109-04-07,2191-02-07,NaN,2191-02-07 00:00:00,1
17,9504,10045,F,2061-03-25,2129-12-01,2129-12-01 00:00:00,2129-12-01 00:00:00,1
33,9550,10094,M,1880-02-29,2180-03-20,2180-03-20 00:00:00,2180-03-20 00:00:00,1
36,9557,10102,M,2035-04-13,2105-06-11,2105-06-11 00:00:00,2105-06-11 00:00:00,1
59,30946,40595,F,2068-03-04,2144-10-31,2144-10-31 00:00:00,2144-10-31 00:00:00,1
66,31311,41976,M,2136-07-28,2202-12-05,NaN,2202-12-05 00:00:00,1
99,31872,44228,F,2112-10-22,2171-04-14,2171-04-14 00:00:00,NaN,1


In [171]:
# row with index 33 caused an overflow error, so we drop it
cd_patients_remove = cd_patients.drop(33)
cd_patients_remove["age_in_days_at_time_of_death"] = (cd_patients_remove["dod"] - cd_patients_remove["dob"]).dt.days
summary_age_days = cd_patients_remove["age_in_days_at_time_of_death"].describe()
print(summary_age_days)
cd_patients_remove

count        6.000000
mean     25699.333333
std       2974.233122
min      21358.000000
25%      24448.000000
50%      25356.500000
75%      27405.750000
max      29891.000000
Name: age_in_days_at_time_of_death, dtype: float64


,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,age_in_days_at_time_of_death
15,9502,10043,M,2109-04-07,2191-02-07,NaN,2191-02-07 00:00:00,1,29891
17,9504,10045,F,2061-03-25,2129-12-01,2129-12-01 00:00:00,2129-12-01 00:00:00,1,25087
36,9557,10102,M,2035-04-13,2105-06-11,2105-06-11 00:00:00,2105-06-11 00:00:00,1,25626
59,30946,40595,F,2068-03-04,2144-10-31,2144-10-31 00:00:00,2144-10-31 00:00:00,1,27999
66,31311,41976,M,2136-07-28,2202-12-05,NaN,2202-12-05 00:00:00,1,24235
99,31872,44228,F,2112-10-22,2171-04-14,2171-04-14 00:00:00,NaN,1,21358
